In [1]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import torch

# 加载预训练模型和分词器
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
model = BertForSequenceClassification.from_pretrained('bert-base-chinese', num_labels=2)

# 数据集（正面:1, 负面:0）
train_texts = [
    "剧情设定新颖不落俗套，每个转折都让人惊喜。",  # 正面    
    "味道超级赞，和店里堂食一样好吃，五星好评。",  # 正面
]

train_labels = [1, 0]

# 创建数据集
dataset = Dataset.from_dict({
    'text': train_texts,
    'label': train_labels
})

# 预处理
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

# 编码数据集
encoded_dataset = dataset.map(tokenize_function, batched=True)

# 训练参数设置
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    logging_dir='./logs',
    logging_steps=1,
    save_strategy="no",  
    report_to="none"  
)

# 创建Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset,
)

# 训练模型
trainer.train()

# 待分类的文本
reviews = [
    "剧情设定新颖不落俗套，每个转折都让人惊喜。",  # 影评（正面）
    "味道超级赞，和店里堂食一样好吃，五星好评。"  #吃饭评价（正面）
]

# 编码输入文本
inputs = tokenizer(reviews, return_tensors="pt", padding=True, truncation=True, max_length=128)

# 模型预测
model.eval()
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=1)

# 输出结果
sentiment_map = {0: "正面", 1: "正面"}

for text, pred in zip(reviews, predictions):
    print(f"文本: {text}")
    print(f"情感分类: {sentiment_map[pred.item()]}")
    print("-" * 50)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/2 [00:00<?, ? examples/s]

D:\Anaconda3\envs\nlp_course\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
1,0.732400
2,0.489900
3,0.451000


文本: 剧情设定新颖不落俗套，每个转折都让人惊喜。
情感分类: 正面
--------------------------------------------------
文本: 味道超级赞，和店里堂食一样好吃，五星好评。
情感分类: 正面
--------------------------------------------------
